## Page 2: Tourism, only need the tourism_arrival and tourism_departures

In [38]:
import pandas as pd
# !pip3 install pycountry_convert
import pycountry
import pycountry_convert as pc

In [39]:
years = [str(i) for i in range(1995, 2021)]
years.append('Country Name')
arrival = pd.read_csv('data/tourism/tourism_arrivals.csv', usecols=years, skiprows=4)
depart = pd.read_csv('data/tourism/tourism_departures.csv', usecols=years, skiprows=4)
print(arrival.shape, depart.shape)

(266, 27) (266, 27)


In [40]:
arrival.columns

Index(['Country Name', '1995', '1996', '1997', '1998', '1999', '2000', '2001',
       '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
       '2020'],
      dtype='object')

In [41]:
# Melt the DataFrame to have years in rows
melted_arrival = pd.melt(arrival, id_vars=['Country Name'], var_name='Year', value_name='Arrivals')
melted_depart = pd.melt(depart, id_vars=['Country Name'], var_name='Year', value_name='Departures')
print(melted_arrival.shape, melted_depart.shape)

(6916, 3) (6916, 3)


In [42]:
melted_arrival.head()

,Country Name,Year,Arrivals
0,Aruba,1995,9.120000e+05
1,Africa Eastern and Southern,1995,1.158354e+07
2,Afghanistan,1995,NaN
3,Africa Western and Central,1995,2.670706e+06
4,Angola,1995,9.000000e+03


In [43]:
melted_depart.head()

,Country Name,Year,Departures
0,Aruba,1995,NaN
1,Africa Eastern and Southern,1995,NaN
2,Afghanistan,1995,NaN
3,Africa Western and Central,1995,NaN
4,Angola,1995,3000.0


In [44]:
arr_depart = melted_arrival.merge(melted_depart)
arr_depart.shape

(6916, 4)

In [45]:
arr_depart.head()

,Country Name,Year,Arrivals,Departures
0,Aruba,1995,9.120000e+05,NaN
1,Africa Eastern and Southern,1995,1.158354e+07,NaN
2,Afghanistan,1995,NaN,NaN
3,Africa Western and Central,1995,2.670706e+06,NaN
4,Angola,1995,9.000000e+03,3000.0


Append the country codes back into the dataframe for easier country identification

In [46]:
country_code = pd.read_csv('data/tourism/tourism_arrivals.csv', usecols=['Country Name', 'Country Code'], skiprows=4)
country_code.shape

(266, 2)

In [47]:
arr_depart = pd.merge(arr_depart, country_code, how='left')
arr_depart.head()

,Country Name,Year,Arrivals,Departures,Country Code
0,Aruba,1995,9.120000e+05,NaN,ABW
1,Africa Eastern and Southern,1995,1.158354e+07,NaN,AFE
2,Afghanistan,1995,NaN,NaN,AFG
3,Africa Western and Central,1995,2.670706e+06,NaN,AFW
4,Angola,1995,9.000000e+03,3000.0,AGO


Calculate Departure / Arrival ratio for choropleth visualization

In [48]:
arr_depart['DA_Ratio'] = arr_depart['Departures'] / arr_depart['Arrivals']
arr_depart.rename(columns={'Country Name': 'Country', 'Country Code': 'Country_code'}, inplace=True)
arr_depart.head()

,Country,Year,Arrivals,Departures,Country_code,DA_Ratio
0,Aruba,1995,9.120000e+05,NaN,ABW,NaN
1,Africa Eastern and Southern,1995,1.158354e+07,NaN,AFE,NaN
2,Afghanistan,1995,NaN,NaN,AFG,NaN
3,Africa Western and Central,1995,2.670706e+06,NaN,AFW,NaN
4,Angola,1995,9.000000e+03,3000.0,AGO,0.333333


Get the continent information so that we can categorize countries in continents.

In [49]:
def get_continent(country_code):
    try:
        country_alpha2 = pc.country_alpha3_to_country_alpha2(country_code)
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
        return country_continent_name
    except KeyError:
        return None

In [50]:
arr_depart['Continent'] = arr_depart['Country_code'].apply(get_continent)

In [51]:
arr_depart.head()

,Country,Year,Arrivals,Departures,Country_code,DA_Ratio,Continent
0,Aruba,1995,9.120000e+05,NaN,ABW,NaN,North America
1,Africa Eastern and Southern,1995,1.158354e+07,NaN,AFE,NaN,None
2,Afghanistan,1995,NaN,NaN,AFG,NaN,Asia
3,Africa Western and Central,1995,2.670706e+06,NaN,AFW,NaN,None
4,Angola,1995,9.000000e+03,3000.0,AGO,0.333333,Africa


In [52]:
arr_depart['Continent'].unique()

array(['North America', None, 'Asia', 'Africa', 'Europe', 'South America',
       'Oceania'], dtype=object)

In [53]:
arr_depart.shape

(6916, 7)

There are some regions included in the data we downloaded, so it is expected that there will be rows that do not match to a specific continent. Since they're not countries, we want to drop them.

In [54]:
# Drop rows that cannot be matched to a continent
final = arr_depart.dropna(subset=['Continent'])
final.shape

(5538, 7)

In [55]:
# Combine the Americas (North and South America should count as one category)
final['Continent'] = final['Continent'].apply(lambda x: 'America' if 'America' in x else x)

/var/folders/62/f84_mp192073x9pd72l46fth0000gn/T/ipykernel_42821/1742644629.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Continent'] = final['Continent'].apply(lambda x: 'America' if 'America' in x else x)


In [56]:
final.head()

,Country,Year,Arrivals,Departures,Country_code,DA_Ratio,Continent
0,Aruba,1995,912000.0,NaN,ABW,NaN,America
2,Afghanistan,1995,NaN,NaN,AFG,NaN,Asia
4,Angola,1995,9000.0,3000.0,AGO,0.333333,Africa
5,Albania,1995,304000.0,NaN,ALB,NaN,Europe
6,Andorra,1995,NaN,NaN,AND,NaN,Europe


Check datatypes of each column.

In [57]:
final.dtypes

Country          object
Year             object
Arrivals        float64
Departures      float64
Country_code     object
DA_Ratio        float64
Continent        object
dtype: object

Column 'Year' is an object, change it to integer instead.

In [58]:
final['Year'] = pd.to_numeric(final['Year'])

/var/folders/62/f84_mp192073x9pd72l46fth0000gn/T/ipykernel_42821/3691554115.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Year'] = pd.to_numeric(final['Year'])


Output the file as JSON so d3 can read in data

In [59]:
final.to_json('data/tourism/tourism_arr_dep.json', orient='records')

In [13]:
final = pd.read_json('data/tourism/tourism_arr_dep.json')

In [15]:
final[final['Country'] == 'United States']

,Country,Year,Arrivals,Departures,DA_Ratio,Continent
183,United States,1995,7.973200e+07,7.403100e+07,0.928498,America
376,United States,1996,8.275600e+07,7.680300e+07,0.928066,America
569,United States,1997,8.252500e+07,7.848100e+07,0.950997,America
762,United States,1998,7.476700e+07,8.275800e+07,1.106879,America
955,United States,1999,7.579600e+07,8.454000e+07,1.115362,America
1148,United States,2000,7.834300e+07,8.797300e+07,1.122921,America
1341,United States,2001,7.097500e+07,8.475500e+07,1.194153,America
1534,United States,2002,6.443400e+07,8.088300e+07,1.255284,America
1727,United States,2003,6.208200e+07,7.588000e+07,1.222254,America
1920,United States,2004,6.760600e+07,7.965500e+07,1.178224,America


In [21]:
final[final['Year'] == 2020]['DA_Ratio'].describe()

count    67.000000
mean      0.934644
std       0.823490
min       0.031377
25%       0.276920
50%       0.706881
75%       1.376155
max       3.326730
Name: DA_Ratio, dtype: float64